In [7]:
import os

# Your desired directory path
desired_path = "c:/Projects/taxi_demand_predictory/"

# Change the current working directory to the specified path
os.chdir(desired_path)

# Verify by printing the current working directory
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  c:\Projects\taxi_demand_predictory


In [8]:

import warnings
warnings.filterwarnings("ignore")

In [9]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-29,1,0.0
1,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,0.0,2.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,2023-01-30,1,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,2.0,0.0,1.0,0.0,0.0,0.0,2023-01-31,1,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-02-01,1,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2023-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40540,14.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,...,4.0,8.0,5.0,4.0,11.0,3.0,8.0,2023-06-26,265,5.0
40541,2.0,1.0,2.0,4.0,1.0,0.0,2.0,2.0,0.0,2.0,...,8.0,3.0,9.0,5.0,3.0,5.0,7.0,2023-06-27,265,3.0
40542,2.0,3.0,2.0,0.0,1.0,1.0,0.0,2.0,0.0,5.0,...,8.0,6.0,2.0,7.0,7.0,8.0,3.0,2023-06-28,265,4.0
40543,0.0,5.0,0.0,2.0,0.0,1.0,1.0,3.0,5.0,7.0,...,15.0,11.0,3.0,2.0,1.0,8.0,7.0,2023-06-29,265,9.0


In [10]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2023, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 674)
y_train.shape=(32595,)
X_test.shape=(7950, 674)
y_test.shape=(7950,)


In [11]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [12]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2023-09-16 11:07:25,929] A new study created in memory with name: no-name-768b19b1-bfad-4be2-a5de-89f0b0c65d2f


[LightGBM] [Warning] feature_fraction is set=0.649524651897823, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.649524651897823
[LightGBM] [Warning] bagging_fraction is set=0.5064827773823755, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5064827773823755
[LightGBM] [Warning] feature_fraction is set=0.649524651897823, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.649524651897823
[LightGBM] [Warning] bagging_fraction is set=0.5064827773823755, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5064827773823755


[I 2023-09-16 11:07:30,831] Trial 0 finished with value: 3.7480697669515752 and parameters: {'num_leaves': 32, 'feature_fraction': 0.649524651897823, 'bagging_fraction': 0.5064827773823755, 'min_child_samples': 8}. Best is trial 0 with value: 3.7480697669515752.


[LightGBM] [Warning] feature_fraction is set=0.9907282588061033, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9907282588061033
[LightGBM] [Warning] bagging_fraction is set=0.3759591177117275, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3759591177117275
[LightGBM] [Warning] feature_fraction is set=0.9907282588061033, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9907282588061033
[LightGBM] [Warning] bagging_fraction is set=0.3759591177117275, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3759591177117275


[I 2023-09-16 11:07:41,575] Trial 1 finished with value: 5.129709602722849 and parameters: {'num_leaves': 205, 'feature_fraction': 0.9907282588061033, 'bagging_fraction': 0.3759591177117275, 'min_child_samples': 96}. Best is trial 0 with value: 3.7480697669515752.


...

[I 2023-09-16 11:07:51,703] Trial 2 finished with value: 5.165543647109014 and parameters: {'num_leaves': 112, 'feature_fraction': 0.7150548737701665, 'bagging_fraction': 0.7396191370530623, 'min_child_samples': 96}. Best is trial 0 with value: 3.7480697669515752.


[LightGBM] [Warning] feature_fraction is set=0.6994885218816154, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6994885218816154
[LightGBM] [Warning] bagging_fraction is set=0.48248153902367197, subsample=1.0 will be ignored. Current value: bagging_fraction=0.48248153902367197
[LightGBM] [Warning] feature_fraction is set=0.6994885218816154, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6994885218816154
[LightGBM] [Warning] bagging_fraction is set=0.48248153902367197, subsample=1.0 will be ignored. Current value: bagging_fraction=0.48248153902367197


[I 2023-09-16 11:07:59,866] Trial 3 finished with value: 3.66749266750607 and parameters: {'num_leaves': 70, 'feature_fraction': 0.6994885218816154, 'bagging_fraction': 0.48248153902367197, 'min_child_samples': 6}. Best is trial 3 with value: 3.66749266750607.


[LightGBM] [Warning] feature_fraction is set=0.3279286601302259, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3279286601302259
[LightGBM] [Warning] bagging_fraction is set=0.5278831881994003, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5278831881994003
[LightGBM] [Warning] feature_fraction is set=0.3279286601302259, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3279286601302259
[LightGBM] [Warning] bagging_fraction is set=0.5278831881994003, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5278831881994003


[I 2023-09-16 11:08:07,980] Trial 4 finished with value: 3.8899255258258885 and parameters: {'num_leaves': 191, 'feature_fraction': 0.3279286601302259, 'bagging_fraction': 0.5278831881994003, 'min_child_samples': 19}. Best is trial 3 with value: 3.66749266750607.


In [13]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 70, 'feature_fraction': 0.6994885218816154, 'bagging_fraction': 0.48248153902367197, 'min_child_samples': 6}


In [14]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.6994885218816154, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6994885218816154
[LightGBM] [Warning] bagging_fraction is set=0.48248153902367197, subsample=1.0 will be ignored. Current value: bagging_fraction=0.48248153902367197


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001F57EE394C0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.48248153902367197,
                               feature_fraction=0.6994885218816154,
                               min_child_samples=6, num_leaves=70))])

In [15]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=2.6243


In [16]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=2979,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [17]:
plot_one_sample(
    example_id=3979,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)